In [138]:
import os

In [139]:
%cd "c:\\Users\\91630\\Desktop\\Devs\\ML\\Prediction_project\\research"


c:\Users\91630\Desktop\Devs\ML\Prediction_project\research


In [140]:
%pwd

'c:\\Users\\91630\\Desktop\\Devs\\ML\\Prediction_project\\research'

In [141]:
os.chdir("../")

In [142]:
%pwd

'c:\\Users\\91630\\Desktop\\Devs\\ML\\Prediction_project'

In [143]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL :str
    local_data_file : Path
    unzip_dir :Path 

In [144]:
from src.ml_project.constants import *
from src.ml_project.utils.common import read_yaml, create_directories

In [145]:
%pip install python-box==6.0.2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [146]:
class configurationManager:
    def __init__(self,
         config_filepath= CONFIG_FILE_PATH 
,params_filepath= PARAMS_FILE_PATH ,
schema_filepath=SCHEMA_FILE_PATH ) :
        self.config = read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config
    

In [147]:
import os
import urllib.request as request
import zipfile
from src.ml_project.logger import logging
from src.ml_project.exception import CustomException
from src.ml_project.utils.common import get_size

In [148]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [149]:
try:
    config = configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e